In [1]:
# Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [2]:
# Generating a corpus of words with which we can train our model

# Dataset from which we will generate our word corpus
word_corpus_df = pd.read_csv("/kaggle/input/part-of-speech-tagging/words_pos.csv")
word_corpus_df.drop(columns = ["Unnamed: 0", "pos_tag"], inplace = True)
print(word_corpus_df.shape)
word_corpus_df.head(10)

(370100, 1)


,word
0,aa
1,aaa
2,aah
3,aahed
4,aahing
5,aahs
6,aal
7,aalii
8,aaliis
9,aals


In [3]:
# Creation of the target dataset that our algoirthms will be performing upon

def create_target_dataset(df, size):
    
    letters = "abcdefghijklmnopqrstuvwxyz"
    words_string = ""
    
    # Creating "size" random words
    for i in range(size):
        
        # Selecting a random word from the word corpus
        random_word_index = random.randint(0, len(df) - 1)
        random_word = df.loc[random_word_index, 'word']
        
        # Determining number of modifications to be done on the randomly selected word
        random_number_of_letter_modifications = 1
        if len(random_word) > 2:
            random_number_of_letter_modifications = random.randint(1, min(int(len(random_word) / 2), 2))

        changed_indices = []
        random_word_list = list(random_word)
        
        # Modifying the randomly selected word
        for _ in range(random_number_of_letter_modifications):
            random_word_index = random.randint(0, len(random_word_list) - 1)
            while random_word_index in changed_indices:
                random_word_index = random.randint(0, len(random_word_list) - 1)
            random_word_index_letter_change = random.choice(letters)
            random_word_list[random_word_index] = random_word_index_letter_change

        random_word = ''.join(random_word_list)
        words_string = words_string + random_word + " "

    return words_string

In [4]:
words_string_levenshtein_distance = create_target_dataset(word_corpus_df, 1000)
words_list_levenshtein_distance = words_string_levenshtein_distance.split()
print(len(words_list_levenshtein_distance))

words_string_symspellpy = create_target_dataset(word_corpus_df, 150000)
words_list_symspellpy = words_string_symspellpy.split()
print(len(words_list_symspellpy))

words_string_bk_tree = create_target_dataset(word_corpus_df, 1000)
words_list_bk_tree = words_string_bk_tree.split()
print(len(words_list_bk_tree))

1000
150000
1000


# Levenshtein Distance Approach

In [5]:
# Method to calculate levenshtein distance (ld) between two words.
# Example: ld between "aba" and "ab" is 1, ld between "abc" and "aed" is 2.
def levenshtein_distance(word1, word2):
    m, n = len(word1), len(word2)
    
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i][j - 1], dp[i - 1][j]) + 1
    
    return dp[m][n]

In [6]:
# Method to use levenshtein distance across the entire word corpus.
# Determines the best suited word to substitute the input_word with.
def correct_spelling_levenshtein_distance(input_word, word_corpus_df):
    min_distance = float('inf')
    best_correction = input_word
    
    # Finding words that have a absolute difference of length 2 from input_word
    possible_correct_words_corpus = []
    for word in word_corpus_df["word"]:
        if((len(word) >= max(len(input_word) - 2, 0)) and (len(word) <= min(len(input_word) + 2, 31))):
            possible_correct_words_corpus.append(word)
    
    # Calculating levenshtein distance between input_word and our possible words
    # Choosing the word with shortest levenshtein distance wrt input_word
    for word in possible_correct_words_corpus:
        distance = levenshtein_distance(input_word, word)
        
        if(distance < min_distance):
            min_distance = distance
            best_correction = word
           
        if(distance == min_distance):
            if(len(word) == len(input_word)):
                best_correction = word
            
    return best_correction

In [ ]:
# Applying our Levenshtein Distance Approach to all 150000 words
from datetime import datetime

start_time = datetime.now()
for word_index in range(10):
    words_list_levenshtein_distance[word_index] = correct_spelling_levenshtein_distance(words_list_levenshtein_distance[word_index], word_corpus_df)
    
end_time = datetime.now()
counter = int(len(words_list_levenshtein_distance) / 1000)

print("Average Time to Process 1000 Words: " + str((end_time - start_time) / counter))

# SymSpellPy Approach

In [7]:
pip install symspellpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Using Norvig's text dataset to generate frequencies of all words and creating a Word Dictionary
# Adding words from our word corpus that are not present in the Word Dictionary
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('/kaggle/input/word-corpus-text/big.txt').read()))

WORDS_DICT = dict(WORDS)

for word in word_corpus_df["word"]:
    if(word not in WORDS_DICT):
        WORDS_DICT[word] = 1

In [9]:
# Initializing our sym_spell variable according to the Words Dictionary
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell()

for word, word_freq in WORDS_DICT.items():
    sym_spell.create_dictionary_entry(word, word_freq)

In [10]:
# Method to select the best possible word correction using or SymSpell object
def correct_spelling_symspell(input_word):
    
    if input_word is None:
        return input_word
    
    suggestions = sym_spell.lookup(input_word, Verbosity.CLOSEST, max_edit_distance = 2, include_unknown=True)
    best_correction = input_word
    best_correction_freq = -1
    min_distance = float('inf')
    for suggestion in suggestions:
        if(suggestion.term in WORDS_DICT):
            distance = levenshtein_distance(input_word, suggestion.term)
            if distance < min_distance:
                min_distance = distance
                best_correction = suggestion.term
                best_correction_freq = WORDS_DICT[suggestion.term]
            if distance == min_distance:
                if best_correction_freq < WORDS_DICT[suggestion.term]:
                    best_correction = suggestion.term
                    best_correction_freq = WORDS_DICT[suggestion.term]
    return best_correction

In [11]:
# Applying our SymSpellPy Approach to all 150000 target words
from datetime import datetime

start_time = datetime.now()
for word_index in range(len(words_list_symspellpy)):
    words_list_symspellpy[word_index] = correct_spelling_symspell(words_list_symspellpy[word_index])
    
end_time = datetime.now()
counter = int(len(words_list_symspellpy) / 1000)
    
print(words_list_symspellpy[:10])
print("Average Time to Process 1000 Words: " + str((end_time - start_time) / counter))

['woodfish', 'dialonian', 'selfheal', 'jadelike', 'communital', 'activital', 'nearliest', 'imprisoning', 'metanotions', 'fainly']
Average Time to Process 1000 Words: 0:00:00.489302


In [12]:
# Determining the accuracy of SymSpellPy Approach

correct_words = 0
wrong_words = 0
for word in words_list_symspellpy:
    if(word in WORDS_DICT):
        correct_words = correct_words + 1
    else:
        wrong_words = wrong_words + 1

print("Correct Words Percentage: " + str((correct_words / len(words_list_symspellpy)) * 100))
print("Wrong Words Percentage: " + str((wrong_words / len(words_list_symspellpy)) * 100))

Correct Words Percentage: 100.0
Wrong Words Percentage: 0.0


# BK Tree + Levenshtein Distance Approach

In [13]:
# BK Tree class for fast work lookup whilst doing spell checking
class BKNode:
    def __init__(self, word):
        self.word = word
        self.children = {}

class BKTree:
    def __init__(self):
        self.root = None
    
    # Method to insert the node into the BK Treee
    def insert(self, word):
        if not self.root:
            self.root = BKNode(word)
            return
        
        # According to the levenshtein distance place the word
        node = self.root
        distance = levenshtein_distance(word, node.word)
        while distance in node.children:
            node = node.children[distance]
            distance = levenshtein_distance(word, node.word)
        
        node.children[distance] = BKNode(word)
    
    # Method to get the list of words having a levenshtein distance of 3 from the input_word
    def query(self, word, max_distance = 3):
        if not self.root:
            return []

        result = []

        def _query(node, distance):
            dist = levenshtein_distance(word, node.word)
            if dist <= max_distance:
                result.append((node.word, dist))
            
            for d in range(dist - max_distance, dist + max_distance + 1):
                child = node.children.get(d)
                if child:
                    _query(child, max_distance)

        _query(self.root, max_distance)
        
        return result

In [14]:
# Building our BK Tree using the word dictionary
bk_tree = BKTree()
for word, word_freq in WORDS_DICT.items():
    bk_tree.insert(word)

In [ ]:
# Applying our BK Tree + Levenshtein Distance Approach to all 150000 target words

from datetime import datetime

start_time = datetime.now()
for word_index in range(len(words_list_bk_tree)):
    suggested_corrections = bk_tree.query(words_list_bk_tree[word_index])
    correct_word = words_list_bk_tree[word_index]
    correct_word_freq = -1
    min_dist = 1000
    for word, ld in suggested_corrections:
        if(min_dist > ld):
            min_dist = ld
            correct_word = word
            correct_word_freq = WORDS_DICT[correct_word]
        if(min_dist == ld):
            if(WORDS_DICT[word] > correct_word_freq):
                correct_word = word
                correct_word_freq = WORDS_DICT[correct_word]
                
    words_list_bk_tree[word_index] = correct_word
    
end_time = datetime.now()
counter = int(len(words_list_bk_tree) / 1000)
    
print("Average Time to Process 1000 Words: " + str((end_time - start_time) / counter))

# FINAL RESULTS

### symspellpy provides the best spelling correction results at: *0.1 to 0.5 seconds average per 1000 words*

## References:
* https://symspellpy.readthedocs.io/en/latest/api/index.html
* https://norvig.com/spell-correct.html